Approach-4<br>
BERT Model Implementation with ELMo embeddings.



In [8]:
#Import required libraries.
! pip install "tensorflow>=1.7.0"
! pip install tensorflow-hub
! pip install pytorch-pretrained-bert pytorch-nlp
! pip install num2words

import bz2
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

import nltk
nltk.download('punkt')

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

exec(open('/content/preprocess_data.py').read())

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

     |████████████████████████████████| 102kB 3.8MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'Tesla K80'

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [10]:
sample_data0 = pd.read_csv('/content/sample.csv')
sample_data = sample_data0.sample(frac=0.1, replace=True, random_state=1)
sample_data = sample_data.reset_index(drop=True)

In [11]:
sample_data.head()

,Class,Title,Article
0,2,Backman out as D-Backs manager,The Diamondbacks will replace Wally Backman as...
1,2,Stanford's Cubit Hired As W. Mich. Coach (AP),AP - Stanford offensive coordinator Bill Cubit...
2,1,An Old Church's New Tilt Inspires Tourists and...,A humble church has something Germany's glorio...
3,4,Record Breaking Supercomputer Performance,US Secretary of Energy Spencer Abraham announc...
4,2,Defrocked Priest Gets Suspended Sentence for M...,A defrocked Irish priest who attacked the lead...


In [12]:
for i in range(len(sample_data['Article'])):
  sample_data['Article'][i] = preprocess(sample_data['Article'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
sample_data.head()

,Class,Title,Article
0,2,Backman out as D-Backs manager,the diamondbacks will replace wally backman a...
1,2,Stanford's Cubit Hired As W. Mich. Coach (AP),ap stanford offensive coordinator bill cubit ...
2,1,An Old Church's New Tilt Inspires Tourists and...,a humble church has something germanys glorio...
3,4,Record Breaking Supercomputer Performance,us secretary of energy spencer abraham announ...
4,2,Defrocked Priest Gets Suspended Sentence for M...,a defrocked irish priest who attacked the lea...


In [14]:
label_check = {1 : 0, 2: 1, 3 : 2, 4 : 3}

In [16]:
# Create sentence and label lists
sentences = sample_data['Article']
labels = [label_check[c] for c in sample_data['Class']]

In [17]:
labels=labels[:50]

In [18]:
sentences.shape

(760,)

**Generating vectors for sentences using ELMo**

In [19]:
tf.compat.v1.disable_eager_execution()
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [20]:
# Extract ELMo features 
embeddings = elmo(sentences, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([760, 137, 1024])

In [21]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(tf.compat.v1.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.compat.v1.reduce_mean(embeddings,1))

In [22]:
# To save computational resources (memory) we split data set into batches of 10 samples each. 
# Then, pass these batches sequentially to the function elmo_vectors( ).
list_articles = [sentences[i:i+10] for i in range(0,50,10)]

In [23]:
list_articles

[0     the diamondbacks will replace wally backman a...
 1     ap stanford offensive coordinator bill cubit ...
 2     a humble church has something germanys glorio...
 3     us secretary of energy spencer abraham announ...
 4     a defrocked irish priest who attacked the lea...
 5     new york reuters international business machi...
 6     ichiro madritsch and cabrera are having a bla...
 7     november three two thousand and four idg news...
 8     london british police charged eight terrorist...
 9     well not the world maybe but possibly your la...
 Name: Article, dtype: object,
 10     researchers are developing tiny atomic clocks...
 11     quot over the past two and a half years the f...
 12     manchester united criticized fans who disrupt...
 13     uk fire equipment manufacturer kidde agrees a...
 14     though coach herman edwards ruled defensive e...
 15     the federal reserve is expected to nudge inte...
 16     the america online unit of time warner inc tw...
 17     st

In [24]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x) for x in list_articles]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
#concatenate the vectors of each word in a single sentence
input_ids = np.concatenate(elmo_train, axis = 0)

In [26]:
input_ids[0]

array([ 0.3831859 , -0.197392  , -0.0324713 , ..., -0.02443597,
        0.25400704,  0.09058411], dtype=float32)

In [27]:
len(labels)

50

In [28]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2021, test_size=0.2)

In [29]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs).to(torch.int64)
validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
train_labels = torch.tensor(np.array(train_labels))
validation_labels = torch.tensor(np.array(validation_labels))

# Select a batch size for training. 
batch_size = 1

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs,train_labels)# train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs,validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [30]:
train_inputs.size()

torch.Size([40, 1024])

In [31]:
train_dataloader

In [32]:
# torch.cuda.empty_cache()
import torch
torch.cuda.empty_cache()

In [33]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 4)
model.cuda()

100%|██████████| 407873900/407873900 [00:14<00:00, 28032514.50B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [34]:
# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)


In [35]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [36]:
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 2

# BERT training loop
for _ in trange(epochs, desc="Epoch"):
  
  ## TRAINING
  # Set our model to training mode
  
  model.train()
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_labels = batch 
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    # loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels) ======= vuthej
    loss = model(b_input_ids, token_type_ids=None, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  # VALIDATION

  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None)
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()